In [9]:
using Pkg
using Plots
using DifferentialEquations
using ParameterizedFunctions
using CSV
using DataFrames
using ODE

niveauATP=0
for ATP_demand in [1e-6] #, 2.495e-6, 1e-5 an array
    niveauATP=niveauATP+1

function equadiff(du,u,t)
  Lac,Glucose,Oxygen,Pyruvate,H, pH,ATP = u
  Glycolyse =  (UptakeMAXGlu*(pgo / ( pgo + Oxygen)) * (maxima * exp(-( pH - kph)^2 / ( 2 * ngaussian^2) ) ) )*( (Glucose)/(Km + Glucose))
  UPTAKE_OXYGEN=(3.0/17.0)*max(0.0,ATP_target-ATP)*(Oxygen/(Kox+Oxygen))
  OXPHOS = min(17.0*Pyruvate, (17.0/3.0)*UPTAKE_OXYGEN)
  UPTAKE_LAC= (UptakeMAXLac  * (1 - ( (pH^n)/(KpH^n + pH^n))) * (Lac/(KLac + Lac)))
  EXCRETION_LAC=max(0,Pyruvate+2*Glycolyse - (1/17) * OXPHOS + UPTAKE_LAC-Pyruvate_target)
  ATP_UTILISATION=ATP_demand#+ ATP_demand * sin(Frequency*t)/InvAmplitude
  du[1] = dLac = -UPTAKE_LAC  + EXCRETION_LAC
  du[2] = dGlucose = -Glycolyse
  du[3] = dOxygen = -(3/17)OXPHOS 
  du[4] = dPyruvate = 2*Glycolyse - (1/17) * OXPHOS + UPTAKE_LAC - EXCRETION_LAC
  du[5] = dH = dLac
  u[6] = pH = -log10(max(0.00000000000000001,H)/1000)
  du[7] =  dATP = 2*Glycolyse + OXPHOS - ATP_UTILISATION
end
    
UptakeMAXLac = 1
KpH = 6.9
n=67.95
KLac = 0.04

UptakeMAXGlu=1e-7
Km=0.04
pgo=0.24
maxima=1.3
kph=7.5
ngaussian=0.27

Pyruvate_target=0.120000000000
poatp = 1
ATP_target = 2.8
Kox = 0.0046

Frequency=0.1
InvAmplitude=5

u0 = [0.0;5.0;0.1;0.12;5.01187234e-5;7.3;2.8]
tspan = (0.0,648000.0)
p = [7.3]


prob = ODEProblem(equadiff,u0,tspan)

#sol = solve(prob,AutoTsit5(Rosenbrock23()),reltol=1e-5,abstol=1e-5, dtmax=1)
sol = solve(prob,AutoTsit5(Rosenbrock23(autodiff=false)),reltol=1e-8,abstol=1e-8, dtmax=1)
pyplot()

Dataframe=DataFrame(Time=sol.t, Lac=sol[1,:], GLu=sol[2,:], Ox=sol[3,:], Pyr=sol[4,:], H=sol[5,:], pH=sol[6,:], ATP=sol[7,:])
CSV.write(string("simu",niveauATP,".csv"), Dataframe)
end

MethodError: MethodError: no method matching iterate(::DiffEqBase.NullParameters)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:589
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:589
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...